In [1]:
from gsample import *

d:\python35\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
img_size = 256
batch_size = 8
train_path = 'E:/kaggle/2018data/stage1_train/'
test_path = 'E:/kaggle/2018data/stage1_test/'
import utils

In [3]:
X_train, Y_train, X_test, sizes_test, train_id = make_df(train_path, test_path, img_size)

In [6]:
from utils import *
for i in range(670):
    bbox = extract_bboxes(Y_train[i])
    for j in range(Y_train[i].shape[-1]):
        tmp_bbox = bbox[j]
        if tmp_bbox[0]==tmp_bbox[2] and tmp_bbox[1]==tmp_bbox[3]:
            print("fuck ",i)

fuck  69
fuck  225
fuck  340
fuck  340


In [7]:
train_id[340]

'7f38885521586fc6011bef1314a9fb2aa1e4935bd581b2991e1d963395eab770'

In [5]:
class shapesDataset(utils.Dataset):
    def __init__(self, image, mask, num):
        self.image = list(image)
        self.mask = list(mask)
        self.count = num
        #self.class_info = [{"source": "", "id": 0, "name": "BG"}]
        self._image_ids = []
        self.image_info = []
        # Background is always the first class
        self.class_info = [{"source": "", "id": 0, "name": "BG"}]
        self.source_class_ids = {}
        self.height = 256
        self.width = 256

    def load_kernel(self, count, height, width):
        """Generate the requested number of synthetic images.
        count: number of images to generate.
        height, width: the size of the generated images.
        """
        # Add classes
        self.height,self.width = height,width
        self.add_class("shapes", 1, "kernel")

        for i in range(count):
            #bg_color, shapes = self.random_image(height, width)
            self.add_image("shapes", image_id=i, path=None,
                           width=width, height=height)

    def load_image(self, image_id):
        return np.array(self.image[image_id])

    def image_reference(self, image_id):
        """Return the shapes data of the image."""
        info = self.image_info[image_id]
        if info["source"] == "shapes":
            return info["shapes"]
        else:
            super(self.__class__).image_reference(self, image_id)

    def load_mask(self, image_id):
        mask_num = self.mask[image_id].shape[2]
        return np.array(self.mask[image_id]),np.ones((mask_num,),dtype=int)

In [6]:
import os
import sys
import random
import math
import re
import time
import numpy as np
import cv2
import matplotlib
import matplotlib.pyplot as plt

from config import Config
import utils
import model as modellib
import visualize
from model import log
import skimage
from data import *
from gsample import *
import scipy.io as sio

class shapesConfig(Config):
    """Configuration for training on the toy shapes dataset.
    Derives from the base Config class and overrides values specific
    to the toy shapes dataset.
    """
    # Give the configuration a recognizable name
    NAME = "shapes"
    STEPS_PER_EPOCH = 100
    # Train on 1 GPU and 8 images per GPU. We can put multiple images on each
    # GPU because the images are small. Batch size is 8 (GPUs * images/GPU).
    GPU_COUNT = 8
    IMAGES_PER_GPU = 8

    # Number of classes (including background)
    NUM_CLASSES = 1 + 1  # background + 1 shapes

    # Use small images for faster training. Set the limits of the small side
    # the large side, and that determines the image shape.
    #IMAGE_MIN_DIM = 128#not sure?
    #IMAGE_MAX_DIM = 128

class InferenceConfig(shapesConfig):
    GPU_COUNT = 8
    IMAGES_PER_GPU = 8

In [7]:
config = shapesConfig()
config.display()
data_train = shapesDataset(X_train,Y_train,len(X_train))
data_train.load_kernel(len(X_train),img_size,img_size)
data_train.prepare()
data_val = shapesDataset(X_train,Y_train,len(X_train))
data_val.load_kernel(len(X_train),img_size,img_size)
data_val.prepare()
#data_val = shapesDataset(xval,yval,len(xval))
#data_val.load_kernel(len(xval),img_size,img_size)
#data_val.prepare()


Configurations:
BACKBONE_SHAPES                [[256 256]
 [128 128]
 [ 64  64]
 [ 32  32]
 [ 16  16]]
BACKBONE_STRIDES               [4, 8, 16, 32, 64]
BATCH_SIZE                     64
BBOX_STD_DEV                   [0.1 0.1 0.2 0.2]
DETECTION_MAX_INSTANCES        100
DETECTION_MIN_CONFIDENCE       0.7
DETECTION_NMS_THRESHOLD        0.3
GPU_COUNT                      8
IMAGES_PER_GPU                 8
IMAGE_MAX_DIM                  1024
IMAGE_MIN_DIM                  800
IMAGE_PADDING                  True
IMAGE_SHAPE                    [1024 1024    3]
LEARNING_MOMENTUM              0.9
LEARNING_RATE                  0.001
MASK_POOL_SIZE                 14
MASK_SHAPE                     [28, 28]
MAX_GT_INSTANCES               100
MEAN_PIXEL                     [123.7 116.8 103.9]
MINI_MASK_SHAPE                (56, 56)
NAME                           shapes
NUM_CLASSES                    2
POOL_SIZE                      7
POST_NMS_ROIS_INFERENCE        1000
POST_NMS_ROIS_TRAINING   

In [8]:
model = modellib.MaskRCNN(mode="training", config=config,
                          model_dir=MODEL_DIR)
model.load_weights(COCO_MODEL_PATH, by_name=True,
                           exclude=["mrcnn_class_logits", "mrcnn_bbox_fc",
                                    "mrcnn_bbox", "mrcnn_mask"])

In [ ]:
model.train(data_train, data_val,
                learning_rate=config.LEARNING_RATE / 10,
                epochs=10,
                layers="heads")


Starting at epoch 0. LR=0.0001

Checkpoint Path: E:\mask_rcnn\logs\shapes20180309T1633\mask_rcnn_shapes_{epoch:04d}.h5
Selecting layers to train
fpn_c5p5               (Conv2D)
fpn_c4p4               (Conv2D)
fpn_c3p3               (Conv2D)
fpn_c2p2               (Conv2D)
fpn_p5                 (Conv2D)
fpn_p2                 (Conv2D)
fpn_p3                 (Conv2D)
fpn_p4                 (Conv2D)
In model:  rpn_model
    rpn_conv_shared        (Conv2D)
    rpn_class_raw          (Conv2D)
    rpn_bbox_pred          (Conv2D)
mrcnn_mask_conv1       (TimeDistributed)
mrcnn_mask_bn1         (TimeDistributed)
mrcnn_mask_conv2       (TimeDistributed)
mrcnn_mask_bn2         (TimeDistributed)
mrcnn_class_conv1      (TimeDistributed)
mrcnn_class_bn1        (TimeDistributed)
mrcnn_mask_conv3       (TimeDistributed)
mrcnn_mask_bn3         (TimeDistributed)
mrcnn_class_conv2      (TimeDistributed)
mrcnn_class_bn2        (TimeDistributed)
mrcnn_mask_conv4       (TimeDistributed)
mrcnn_mask_bn4    

d:\python35\lib\site-packages\tensorflow\python\ops\gradients_impl.py:96: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


Epoch 1/10


In [17]:
path = train_path+train_id[39]
mask = np.zeros((256,256,62))
j=0
for mask_file in next(os.walk(path + '/masks/'))[2]:
            mask_ = cv2.imread(path + '/masks/' + mask_file, 0)
            mask_ = cv2.resize(mask_, (img_size, img_size))
            #mask_ = mask_[:, :, np.newaxis]
            mask[:,:,j] = mask_
            j += 1

In [18]:
bbox = extract_bboxes(mask)

In [19]:
bbox

array([[  3, 170,  20, 194],
       [ 35, 165,  53, 180],
       [ 19, 186,  39, 207],
       [ 88,  88, 114, 112],
       [156, 180, 162, 186],
       [ 57, 208,  76, 236],
       [217,  15, 233,  31],
       [236, 199, 256, 229],
       [ 19, 105,  44, 126],
       [237, 231, 256, 256],
       [195,  66, 216,  86],
       [134, 151, 156, 173],
       [163, 214, 178, 228],
       [234,   6, 246,  26],
       [193, 139, 213, 162],
       [130, 180, 164, 208],
       [251, 167, 256, 179],
       [167, 227, 182, 245],
       [  0, 220,  15, 240],
       [ 13,  83,  35, 103],
       [236, 196, 241, 202],
       [130,  96, 140, 107],
       [ 64, 228,  89, 243],
       [ 44, 189,  64, 209],
       [179, 191, 196, 211],
       [198, 200, 212, 215],
       [  3, 247,  22, 256],
       [ 82, 252,  93, 256],
       [ 35, 225,  56, 245],
       [188, 213, 212, 237],
       [155, 150, 167, 169],
       [134,  80, 159, 103],
       [181, 249, 195, 256],
       [128, 246, 144, 256],
       [ 16, 1